In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
#     'dy_hi', 
#     'dy_lo',
    'dy_ladd'
]

In [3]:
def collect_fux(pars_post, coverage=coverage0, alg="VSC"):    
    m = ModelIntv()
    
    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv={}, polished=True)

        ms = ms.reset_index().assign(Key=i, FollowUp=0, Duration=0)
        mss.append(ms)
        
        ms_stats.update({'Key': i, 'Scenario': 'Baseline', 'FollowUp': 0, 'Duration': 0})
        mss_stats.append(ms_stats)
    
        for n_fu in list(range(7)): # + [24, 30, 36]:
            for dur in np.linspace(0, 4, 9):
                intv = {'FullACF': {'Coverage': coverage, 'FollowUp': n_fu, 'Duration': dur, 'ScreenAlg': alg}}
                
                _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, intv=intv, polished=True)
                
                ms = ms.reset_index().assign(Key=i, FollowUp=n_fu, Duration=dur)
                mss.append(ms)

                ms_stats.update({'Key': i, 'Scenario': 'Intervention', 'FollowUp': n_fu, 'Duration': dur})
                mss_stats.append(ms_stats)
            
    mss = pd.concat(mss).assign(Coverage = coverage)
    mss_stats = pd.DataFrame(mss_stats).assign(Coverage = coverage)
    return mss, mss_stats

In [4]:
for folder in folders:
    print(folder)
    for cov in [0.1, 0.2, 0.5]:
        print(folder, cov)
        
        out_path = f'out/{folder}'
        pars_post = json.load(open(f'{out_path}/Post.json', 'r'))

        sel = rd.choice(list(range(len(pars_post))), n_collect)
        sel.sort()
        pars_post = [pars_post[i] for i in sel]

        mss, mss_stats = collect_fux(pars_post, coverage=cov)
        mss_stats = mss_stats.fillna(0)
        mss.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}.csv')
        mss_stats.to_csv(f'{out_path}/Sim_VulFu_fudur_{cov:.2f}_stats.csv')

dy_ladd
dy_ladd 0.1
dy_ladd 0.2
dy_ladd 0.5


In [5]:
mss_stats

,Inc,Mor,ACF_Footfall,ACF_Screened,ACF_Yield,ACF_TP,ACF_Fl,ACF_Sl,ACF_TPT,ACF_fu_Footfall,...,ACF_Alt_Yield,ACF_Alt_TP,ACF_Alt_Fl,ACF_Alt_Sl,ACF_Alt_TPT,Key,Scenario,FollowUp,Duration,Coverage
0,116908.919531,13027.180504,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Baseline,0,0.0,0.5
1,103167.542968,10584.151882,1.267349e+07,1.265622e+07,31278.003280,14084.670806,14084.670806,0.000000,17193.332473,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.0,0.5
2,103200.144302,10589.705714,1.267349e+07,1.265621e+07,31284.713272,14091.975288,14091.975288,0.000000,17192.737985,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,0.5,0.5
3,103200.338925,10589.709997,1.267349e+07,1.265621e+07,31284.688244,14091.945820,14091.945820,0.000000,17192.742424,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.0,0.5
4,103200.333963,10589.710911,1.267349e+07,1.265621e+07,31284.693072,14091.951447,14091.951447,0.000000,17192.741625,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0,Intervention,0,1.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,146812.300744,12139.354148,1.267349e+07,1.264105e+07,63342.417472,18154.686543,17928.149563,226.536980,45187.730929,1.007177e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,2.0,0.5
12796,146324.125362,12076.460096,1.267349e+07,1.264088e+07,63303.638694,18029.018121,17777.455833,251.562288,45274.620572,1.099374e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,2.5,0.5
12797,145953.298633,12028.507050,1.267349e+07,1.264075e+07,63274.580727,17933.226322,17662.314346,270.911976,45341.354405,1.170439e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,3.0,0.5
12798,145662.720962,11990.789373,1.267349e+07,1.264065e+07,63252.048807,17857.831800,17571.579039,286.252761,45394.217008,1.226842e+07,...,0.0,0.0,0.0,0.0,0.0,199,Intervention,6,3.5,0.5
